# K-Means 활용 가치관점수 분석

- 가치관이 비슷한 유저 추천
- 총평 : 그룹간 인원을 정할 수 없어 활용하기 어려움.
- 결론 : 확인할 필요없습니다!

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# 가치관점수 : 
# 디테일 <-> 속도 : 디테일을 중요시하는지 속도를 중요시하는지
# 과정 <-> 결과 : 과정을 중요시하는지 결과를 중요시하는지
# 창업 <-> 취업 : 창업을 꿈꾸는지 취업을 꿈꾸는지
# 아침 <-> 밤 : 아침형인간인지 밤형 인간인지

# 점수 1-100점으로 랜덤 생성
user_style = pd.DataFrame(np.random.randint(0,101,size=(160,4)) , index=range(1,161), columns=['속도','결과','취업','밤'])
user_style['디테일'] = 100 - user_style['속도']
user_style['과정'] = 100 - user_style['결과']
user_style['창업'] = 100 - user_style['취업']
user_style['아침'] = 100 - user_style['밤']
display(user_style)

,속도,결과,취업,밤,디테일,과정,창업,아침
1,61,34,62,42,39,66,38,58
2,67,22,64,33,33,78,36,67
3,29,68,38,85,71,32,62,15
4,8,42,11,36,92,58,89,64
5,48,30,9,1,52,70,91,99
...,...,...,...,...,...,...,...,...
156,71,25,9,26,29,75,91,74
157,26,26,69,65,74,74,31,35
158,52,75,2,2,48,25,98,98
159,23,34,48,67,77,66,52,33


In [2]:
from sklearn.preprocessing import StandardScaler 

# 표준화 진행
my_scaler = StandardScaler()
user_style_ = my_scaler.fit_transform(user_style)

# k-means로 10개의 군집으로 나눈다.
my_km = KMeans(n_clusters = 10, random_state = 123)
my_km.fit(user_style_)
my_centroids = my_km.cluster_centers_
my_cluster_labels = my_km.labels_             

# 각 클러스터 중심 출력
pd.DataFrame(my_centroids,columns = user_style.columns)

,속도,결과,취업,밤,디테일,과정,창업,아침
0,-1.167986,-0.193087,-0.228061,-0.779143,1.167986,0.193087,0.228061,0.779143
1,0.764962,0.394169,0.464576,1.029100,-0.764962,-0.394169,-0.464576,-1.029100
2,0.727143,0.799685,1.281287,-0.577119,-0.727143,-0.799685,-1.281287,0.577119
3,1.414220,-0.863742,-1.028513,0.631632,-1.414220,0.863742,1.028513,-0.631632
4,0.371902,-0.841653,-0.369677,-0.866305,-0.371902,0.841653,0.369677,0.866305
5,-1.017525,-1.151328,-0.664479,0.650423,1.017525,1.151328,0.664479,-0.650423
6,-0.990748,0.864331,-1.083331,0.406198,0.990748,-0.864331,1.083331,-0.406198
7,0.819787,1.086407,-0.585732,-1.253987,-0.819787,-1.086407,0.585732,1.253987
8,-0.401654,-1.069903,1.435491,-0.531986,0.401654,1.069903,-1.435491,0.531986
9,-0.734650,0.961351,0.659763,0.924233,0.734650,-0.961351,-0.659763,-0.924233


In [3]:
# 각 유저 정보에 어떤 그룹으로 분류되었는지 정보 추가
user_style['cluster_label'] = my_cluster_labels
user_style

,속도,결과,취업,밤,디테일,과정,창업,아침,cluster_label
1,61,34,62,42,39,66,38,58,4
2,67,22,64,33,33,78,36,67,4
3,29,68,38,85,71,32,62,15,9
4,8,42,11,36,92,58,89,64,0
5,48,30,9,1,52,70,91,99,4
...,...,...,...,...,...,...,...,...,...
156,71,25,9,26,29,75,91,74,4
157,26,26,69,65,74,74,31,35,8
158,52,75,2,2,48,25,98,98,7
159,23,34,48,67,77,66,52,33,5


In [4]:
# 주어진 유저가 어떤 스타일인지 확인하는 함수

def checkStyle(user):
    style = []
    if user['디테일'] > user['속도']:
        style.append('디테일')
    else:
        style.append('속도')
    if user['과정'] > user['결과']:
        style.append('과정')
    else:
        style.append('결과')
    if user['창업'] > user['취업']:
        style.append('창업')
    else:
        style.append('취업')
    if user['아침'] > user['밤']:
        style.append('아침')
    else:
        style.append('밤')
    return style

In [5]:
# 각 그룹별 인원을 확인할 수 있도록 새로운 데이터프레임 생성
cluster_user = user_style.pivot_table(index='cluster_label', columns=user_style.index, values='아침')
cluster_user

,1,2,3,4,5,6,7,8,9,10,...,151,152,153,154,155,156,157,158,159,160
cluster_label,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,64.0,NaN,NaN,87.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,94.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,58.0,67.0,NaN,NaN,99.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,76.0,76.0,74.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,...,NaN,61.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,77.0


In [14]:
# 각 그룹을 돌면서
for num in range(0,10):
    print(f"<{num+1}번째 그룹>")
    group_list = []    
    i  = 1
    for check in -cluster_user.iloc[num].isnull():
        # 결측치가 아니면 group_list에 해당 숫자(인덱스)를 추가한다음
        if check:
            group_list.append(i)
        i += 1
    
    # 그룹을 돌면서 해당 그룹에 해당하는 유저에 대해 출력
    for i in group_list:
        # print(user_style.iloc[i-1])
        print(f"{i}번째 유저: {checkStyle(user_style.iloc[i-1])}")

    print("=================================")

<1번째 그룹>
4번째 유저: ['디테일', '과정', '창업', '아침']
7번째 유저: ['디테일', '결과', '창업', '아침']
17번째 유저: ['디테일', '과정', '창업', '아침']
19번째 유저: ['디테일', '결과', '취업', '아침']
31번째 유저: ['디테일', '결과', '취업', '아침']
38번째 유저: ['디테일', '결과', '취업', '밤']
44번째 유저: ['디테일', '결과', '창업', '아침']
52번째 유저: ['디테일', '과정', '창업', '아침']
69번째 유저: ['디테일', '결과', '창업', '아침']
79번째 유저: ['디테일', '과정', '창업', '아침']
128번째 유저: ['디테일', '과정', '취업', '아침']
130번째 유저: ['디테일', '결과', '창업', '아침']
140번째 유저: ['디테일', '결과', '취업', '아침']
148번째 유저: ['디테일', '과정', '취업', '아침']
<2번째 그룹>
12번째 유저: ['속도', '결과', '창업', '밤']
13번째 유저: ['속도', '결과', '창업', '밤']
15번째 유저: ['디테일', '결과', '취업', '밤']
21번째 유저: ['디테일', '과정', '취업', '밤']
34번째 유저: ['속도', '과정', '취업', '밤']
46번째 유저: ['속도', '결과', '창업', '밤']
62번째 유저: ['속도', '결과', '취업', '밤']
64번째 유저: ['속도', '과정', '취업', '밤']
72번째 유저: ['속도', '결과', '창업', '밤']
76번째 유저: ['속도', '결과', '창업', '밤']
81번째 유저: ['속도', '결과', '창업', '밤']
83번째 유저: ['속도', '결과', '취업', '밤']
89번째 유저: ['속도', '결과', '취업', '밤']
91번째 유저: ['디테일', '결과', '창업', '밤']
94번째 유저: ['속도', '결과', '취업'